In [51]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
import sys
sys.path.append("../")
import functions as f
plt.style.use('seaborn-v0_8-whitegrid')
#plt.style.available
import seaborn as sns
import load_data as ld
import classes as cl

In [52]:
X, y = ld.load_simple_data()

X = X.T
inputs = X
targets = y.T

network_input_size = X.shape[1]
layer_output_sizes = [10,10,3]
activation_funcs = [f.sigmoid, f.sigmoid, f.softmax]

layers = f.create_layers_batch(network_input_size, layer_output_sizes)


#predictions = f.feed_forward_batch(inputs, layers, activation_funcs)
predictions = f.train_network(inputs, targets, layers, activation_funcs, 0.001, 2000)
accuracy_score = f.accuracy(predictions, targets)

print(accuracy_score)
print(f.mse(predictions, targets))
print(f.r2(predictions, targets))

# MSE function definition
def MSE(y_data, y_model):
    n = np.size(y_model)
    return np.sum((y_data - y_model)**2) / n

accuracy_score = f.accuracy(predictions, targets)
print(f"Accuracy: {accuracy_score}")

#MSE
mse_value = f.mse(predictions, targets)
print(f"MSE: {mse_value}")

#R2 score
r2_value = f.r2(predictions, targets)
print(f"R2 Score: {r2_value}")



0.79
3.651680127313877
-101.02777636772468
Accuracy: 0.79
MSE: 3.651680127313877
R2 Score: -101.02777636772468
